In [ ]:
import geopandas as gpd
import pandas as pd



# --- Lista dei file e delle categorie corrispondenti ---
files = {
    "../Data/Clean/Analisi/sport_tempo_libero_unito_clean.geojson": "Sport e Tempo Libero",
    "../Data/Clean/Analisi/istruzione_filtrato_unito_clean.geojson": "Istruzione",
    "../Data/Clean/Analisi/sanita_filtrato_unito_clean.geojson": "Sanità",
    "../Data/Clean/Analisi/cultura_spettacolo_filtrato_unito_clean.geojson": "Cultura e Spettacolo"
}

# --- Carico e aggiungo la colonna categoria ---
gdfs = []
for path, categoria in files.items():
    gdf = gpd.read_file(path)
    gdf["Categoria"] = categoria   # nuova colonna con il nome desiderato
    gdfs.append(gdf)

# --- Concateno tutti in un unico GeoDataFrame ---
gdf_unico = pd.concat(gdfs, ignore_index=True)

gdf_unico.to_file("../Data/Clean/Analisi/ciclabili_con_categoria_unite.geojson", driver="GeoJSON")

In [6]:
# --- Creo colonne booleane per ogni categoria ---
categorie = list(files.values())  # ["Sport e Tempo Libero", "Istruzione", "Sanità", "Cultura e Spettacolo"]

for cat in categorie:
    gdf_unico[cat] = gdf_unico["Categoria"] == cat

gdf_unico.to_file("../Data/Clean/Analisi/ciclabili_con_categoria_unite.geojson", driver="GeoJSON")


In [ ]:


# Carico il file unificato
gdf = gpd.read_file("../Data/Clean/Analisi/ciclabili_con_categoria_unite.geojson")

# Raggruppiamo per nome della strada (se vuoi anche per 'geometry' possiamo ragionare dopo)
# Manteniamo una sola geometria (es. la prima) e facciamo OR logico sui booleani
agg_dict = {col: "max" for col in categorie}  # max su boolean = OR logico
agg_dict.update({
    "length": "sum",     # sommiamo le lunghezze
    
})

gdf_grouped = gdf.groupby("name").agg(agg_dict).reset_index()

# Per la geometria: prendiamo la prima (in alternativa possiamo unire con unary_union se ha più pezzi)
geom = gdf.groupby("name").geometry.unary_union().reset_index()

# Merge tra geometrie e attributi aggregati
gdf_final = geom.merge(gdf_grouped, on="name")

# Ricreiamo un GeoDataFrame
gdf_final = gpd.GeoDataFrame(gdf_final, geometry="geometry", crs=gdf.crs)

# Salvo il risultato
gdf_final.to_file("../Data/Clean/Analisi/ciclabili_comuni.geojson", driver="GeoJSON")





In [17]:
gdf = gpd.read_file("../Data/Clean/Analisi/ciclabili_con_categoria_unite.geojson")

categorie = ["Sport e Tempo Libero", "Istruzione", "Sanità", "Cultura e Spettacolo"]

# Aggrego solo i booleani con OR logico, e tengo gli altri campi dalla "prima riga"
agg_dict = {col: "max" for col in categorie}
agg_dict.update({
    "origine": "first",
    "MUNICIPIO": "first"
})

# Raggruppo per geometria
gdf_grouped = gdf.groupby("geometry").agg(agg_dict).reset_index()

# Ricreo GeoDataFrame
gdf_grouped = gpd.GeoDataFrame(gdf_grouped, geometry="geometry", crs=gdf.crs)

# Salvo
gdf_grouped.to_file("../Data/Clean/Analisi/ciclabili_uniche.geojson", driver="GeoJSON")
